In [16]:
# Imports + Configuration
import os
import time
import requests
import pandas as pd
from datetime import datetime, timezone, timedelta

# =========================
# ClickUp Configuration
# =========================
SPACE_ID = "90159483029"
TEAM_ID = "90152198197"  # For template lookups
TARGET_FOLDER_NAME = "50 HTL | Oitilo Mani"
TARGET_LIST_NAME = "HTL | Oitilo Mani Execution"
TASK_TEMPLATE_NAME = "HTL | TEMPLATE | Execution"
# TASK_TEMPLATE_ID = "t-86c7n8hr7"  # Fallback if name lookup fails
CSV_PATH_SIMPLIFIED = "OITILO_ClickUP_WBS_simplified.csv"

CLICKUP_TOKEN = "pk_56660333_WATA0RDNID48ZA30VX30Z2CRSNB16SN3"
DRY_RUN = False
BASE = "https://api.clickup.com/api/v2"

# Authentication
sess = requests.Session()
sess.headers.update({
    "Authorization": CLICKUP_TOKEN,
    "Accept": "application/json",
    "Content-Type": "application/json",
})

In [17]:
# Helper functions (HTTP + time + deps parsing)
def req(method: str, url: str, *, params=None, json=None, ok=(200, 201), retries=6):
    if DRY_RUN:
        print(f"[DRY_RUN] {method} {url} params={params} json={json}")
        return {}
    last = None
    for i in range(retries):
        r = sess.request(method, url, params=params, json=json, timeout=60)
        if r.status_code in ok:
            return r.json() if r.text else {}
        if r.status_code == 429 or r.status_code >= 500:
            wait = min(60, 2 ** i)
            time.sleep(wait)
            last = (r.status_code, r.text[:400])
            continue
        snippet = (r.text or "")[:400]
        raise RuntimeError(f"{method} {url} failed: {r.status_code} {snippet}")
    raise RuntimeError(f"{method} {url} failed after retries. Last error: {last}")

def date_to_ms_midday_utc(date_str: str) -> int:
    d = datetime.strptime(date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc) + timedelta(hours=12)
    return int(d.timestamp() * 1000)

def days_to_ms(days: float) -> int:
    return int(float(days) * 24 * 60 * 60 * 1000)

def split_deps(dep_cell) -> list[str]:
    if dep_cell is None or (isinstance(dep_cell, float) and pd.isna(dep_cell)):
        return []
    s = str(dep_cell).strip()
    if not s or s.lower() == "nan":
        return []
    parts = []
    for chunk in s.replace("；", ";").replace(",", ";").split(";"):
        c = chunk.strip()
        if c:
            parts.append(c)
    return parts

In [18]:
# Main logic: Import tasks from simplified CSV to ClickUp
print("Using token prefix:", (CLICKUP_TOKEN[:6] + "…"))
print("Target: Space ID={}, Folder={}, List={}".format(SPACE_ID, TARGET_FOLDER_NAME, TARGET_LIST_NAME))

# -------- Simplified CSV flow (find folder/list by name, import tasks) --------
# Find folder by name
folders = req("GET", f"{BASE}/space/{SPACE_ID}/folder", params={"archived": "false"}).get("folders", [])
folder_id = None
for f in folders:
    if str(f.get("name", "")).strip().lower() == TARGET_FOLDER_NAME.strip().lower():
        folder_id = str(f.get("id"))
        break
if folder_id is None:
    raise RuntimeError(f"Folder not found: {TARGET_FOLDER_NAME} in space {SPACE_ID}")
print(f"Found Folder: {TARGET_FOLDER_NAME} (id={folder_id})")

# Find list by name
lists = req("GET", f"{BASE}/folder/{folder_id}/list", params={"archived": "false"}).get("lists", [])
list_id = None
for l in lists:
    if str(l.get("name", "")).strip().lower() == TARGET_LIST_NAME.strip().lower():
        list_id = str(l.get("id"))
        break
if list_id is None:
    created = req("POST", f"{BASE}/folder/{folder_id}/list", json={"name": TARGET_LIST_NAME})
    list_id = str(created.get("id"))
print(f"Found List: {TARGET_LIST_NAME} (id={list_id})")

# Get custom fields for the list to find WBS and Branch field IDs
custom_fields = req("GET", f"{BASE}/list/{list_id}/field").get("fields", [])
wbs_field_id = None
branch_field_id = None
branch_options = {}
for field in custom_fields:
    fname = str(field.get("name", "")).strip().lower()
    if fname == "wbs":
        wbs_field_id = str(field.get("id"))
        print(f"Found WBS custom field (id={wbs_field_id})")
    if fname == "branch":
        branch_field_id = str(field.get("id"))
        opts = field.get("type_config", {}).get("options", [])
        branch_options = {
            str(opt.get("name", "")).strip().lower(): opt.get("id")
            for opt in opts if opt.get("id")
        }
        print(f"Found Branch custom field (id={branch_field_id}) with {len(branch_options)} options")

if not wbs_field_id:
    print("⚠ Warning: WBS custom field not found in list. WBS values will not be set.")
if not branch_field_id:
    print("⚠ Warning: Branch custom field not found in list. Branch values will not be set.")

# Find task template by name (using TEAM_ID) - KEY IS "templates" NOT "task_templates"!
template_id = TASK_TEMPLATE_ID
try:
    templates = req("GET", f"{BASE}/team/{TEAM_ID}/taskTemplate").get("templates", [])
    print(f"\nSearching for template: '{TASK_TEMPLATE_NAME}'")
    print(f"Found {len(templates)} templates. Checking...")
    
    for t in templates:
        t_name = str(t.get("name", "")).strip()
        t_id = str(t.get("id", ""))
        
        if t_name.lower() == TASK_TEMPLATE_NAME.strip().lower():
            template_id = t_id
            print(f"  ✓ MATCH FOUND: '{t_name}' (id={t_id})")
            break
    
    if template_id == TASK_TEMPLATE_ID:
        print(f"\n⚠ Template '{TASK_TEMPLATE_NAME}' not found, using fallback: {TASK_TEMPLATE_ID}")
    else:
        print(f"\n✓ Found Template: {TASK_TEMPLATE_NAME} (id={template_id})")
except Exception as e:
    print(f"Could not search templates, using fallback: {TASK_TEMPLATE_ID} ({e})")
print(f"Using Template ID: {template_id}")

# Load CSV
df_s = pd.read_csv(CSV_PATH_SIMPLIFIED)
required = ["WBS", "Branch", "Task Name", "Dependencies", "Estimated Time (days)", "Start Date", "Due Date", "Description"]
missing = [c for c in required if c not in df_s.columns]
if missing:
    raise ValueError(f"CSV missing columns: {missing}. Found: {list(df_s.columns)}")

# Clean data
df_s = df_s.copy()
df_s["WBS"] = df_s["WBS"].astype(str).str.strip()
df_s["Branch"] = df_s["Branch"].astype(str).str.strip()
df_s["Task Name"] = df_s["Task Name"].astype(str).str.strip()
df_s["Dependencies"] = df_s["Dependencies"].astype(str)
df_s["Estimated Time (days)"] = df_s["Estimated Time (days)"].apply(lambda x: None if pd.isna(x) else x)
df_s["Start Date"] = df_s["Start Date"].astype(str).str.strip()
df_s["Due Date"] = df_s["Due Date"].astype(str).str.strip()
df_s["Description"] = df_s["Description"].astype(str).fillna("").str.strip()

# Sort by Start Date, then Task Name
def safe_dt_s(s):
    try:
        return datetime.strptime(s, "%Y-%m-%d")
    except:
        return datetime(2100, 1, 1)

df_s = df_s.sort_values(by=["Start Date", "Task Name"], key=lambda col: col.map(safe_dt_s) if col.name == "Start Date" else col).reset_index(drop=True)

print(f"Loaded {len(df_s)} tasks from {CSV_PATH_SIMPLIFIED}")

# Create tasks from template and update fields
name_to_id_s = {}
created_cnt_s = 0
updated_cnt_s = 0

for _, r in df_s.iterrows():
    task_name = r["Task Name"]
    wbs_value = r["WBS"]
    branch_value = r["Branch"]
    
    created = req(
        "POST",
        f"{BASE}/list/{list_id}/taskTemplate/{template_id}",
        json={"name": task_name},
        ok=(200, 201),
    )
    task_id = str(created.get("id") or created.get("task", {}).get("id") or "")
    if not task_id:
        raise RuntimeError(f"Could not parse created task id for '{task_name}'. Response: {created}")
    
    # Store by task name for dependency lookup
    name_to_id_s[task_name] = task_id
    created_cnt_s += 1

    # Build payload for field updates
    payload = {}
    
    # Start Date
    sd = r["Start Date"]
    if sd and sd.lower() != "nan":
        payload["start_date"] = date_to_ms_midday_utc(sd)
        payload["start_date_time"] = False
    
    # Due Date
    dd = r["Due Date"]
    if dd and dd.lower() != "nan":
        payload["due_date"] = date_to_ms_midday_utc(dd)
        payload["due_date_time"] = False
    
    # Estimated Time (days)
    est = r["Estimated Time (days)"]
    if est is not None and not (isinstance(est, float) and pd.isna(est)):
        try:
            payload["time_estimate"] = days_to_ms(float(est))
        except:
            pass
    
    # # Description - append to template description
    # desc_extra = r.get("Description", "")
    # if desc_extra and desc_extra.lower() != "nan":
    #     base_desc = created.get("description") or created.get("task", {}).get("description") or ""
    #     combined_desc = (base_desc + "\n\n" + desc_extra).strip() if base_desc else desc_extra
    #     payload["description"] = combined_desc
    
    # Update task with all fields
    if payload:
        req("PUT", f"{BASE}/task/{task_id}", json=payload, ok=(200,))
        updated_cnt_s += 1
    
    # Set WBS custom field if field exists and value is valid
    if wbs_field_id and wbs_value and wbs_value.lower() not in ["nan", ""]:
        try:
            req(
                "POST",
                f"{BASE}/task/{task_id}/field/{wbs_field_id}",
                json={"value": wbs_value},
                ok=(200,)
            )
        except Exception as e:
            print(f"  ⚠ Could not set WBS for task '{task_name}': {e}")
    
    # Set Branch custom field if field exists and value is valid
    if branch_field_id and branch_value and branch_value.lower() not in ["nan", ""]:
        opt_id = branch_options.get(branch_value.strip().lower())
        if opt_id:
            try:
                req(
                    "POST",
                    f"{BASE}/task/{task_id}/field/{branch_field_id}",
                    json={"value": opt_id},
                    ok=(200,)
                )
            except Exception as e:
                print(f"  ⚠ Could not set Branch for task '{task_name}': {e}")
        else:
            print(f"  ⚠ Branch value '{branch_value}' not in dropdown options: {list(branch_options.keys())}")

print(f"Created: {created_cnt_s} tasks")
print(f"Updated: {updated_cnt_s} tasks with field values")

# Link dependencies
deps_added_s = 0
deps_missing_s = 0
deps_already_s = 0

for _, r in df_s.iterrows():
    child_name = r["Task Name"]
    child_id = name_to_id_s.get(child_name)
    if not child_id:
        continue
    
    for parent_name in split_deps(r["Dependencies"]):
        parent_id = name_to_id_s.get(parent_name)
        if not parent_id:
            deps_missing_s += 1
            continue
        try:
            req(
                "POST",
                f"{BASE}/task/{child_id}/dependency",
                json={"depends_on": parent_id},
                ok=(200, 201),
            )
            deps_added_s += 1
        except RuntimeError as e:
            if "already" in str(e).lower() or "exist" in str(e).lower():
                deps_already_s += 1
            else:
                raise

print(f"Dependencies linked: {deps_added_s}")
if deps_missing_s > 0:
    print(f"  (Missing parent tasks: {deps_missing_s})")
if deps_already_s > 0:
    print(f"  (Already existed: {deps_already_s})")

print("\n✓ Import complete!")


Using token prefix: pk_566…
Target: Space ID=90159483029, Folder=50 HTL | Oitilo Mani, List=HTL | Oitilo Mani Execution
Found Folder: 50 HTL | Oitilo Mani (id=901513698403)
Found List: HTL | Oitilo Mani Execution (id=901520355458)
Found WBS custom field (id=983e99c7-e559-4e37-96fe-fad10888b21e)
Found Branch custom field (id=db9812ab-6f1e-4b95-a913-37f9dd5ac24d) with 5 options

Searching for template: 'HTL | TEMPLATE | Execution'
Found 12 templates. Checking...
  ✓ MATCH FOUND: 'HTL | TEMPLATE | Execution' (id=t-86c7t51ff)

✓ Found Template: HTL | TEMPLATE | Execution (id=t-86c7t51ff)
Using Template ID: t-86c7t51ff
Loaded 45 tasks from OITILO_ClickUP_WBS_simplified.csv
Created: 45 tasks
Updated: 45 tasks with field values
Dependencies linked: 50
  (Missing parent tasks: 44)

✓ Import complete!
